In [ ]:
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/machineHack/Workation Price Prediction Challenge/Participant_Data_WPPH.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('finish')


finish


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train=pd.read_csv("/content/Participant_Data_WPPH/Train.csv")
test = pd.read_csv('/content/Participant_Data_WPPH/Test.csv')
submission = pd.read_csv('/content/Participant_Data_WPPH/sample submission.csv')

In [ ]:
train['Travel Date'] = pd.to_datetime(train['Travel Date'])

In [ ]:
train['Year']=pd.DatetimeIndex(train['Travel Date']).year
train['month']=pd.DatetimeIndex(train['Travel Date']).month


In [ ]:
train[:5]

,Uniq Id,Package Name,Package Type,Destination,Itinerary,Places Covered,Travel Date,Hotel Details,Start City,Airline,Flight Stops,Meals,Sightseeing Places Covered,Cancellation Rules,Per Person Price,Year,month
0,e788ab76d9d8cf1e6ed2f139645ca5d1,Best of Shimla and Manali Holiday from Delhi,Standard,New Delhi|Shimla|Manali|Chandigarh,1N New Delhi . 2N Shimla . 2N Manali . 1N Chan...,New Delhi|Shimla|Manali|Chandigarh,2021-07-30,Not Available,Mumbai,Not Available,2,3,Not Available,Not Available,11509.0,2021,7
1,178f892630ce3e335a5a41d5d83937fd,Kashmir Valley vacation,Premium,Srinagar|Pahalgam|Srinagar,1N Srinagar . 2N Pahalgam . 1N Srinagar,Srinagar|Pahalgam|Srinagar,2021-08-12,The Orchard Retreat & Spa:4.6|WelcomHotel Pine...,New Delhi,IndiGo|IndiGo,0,5,Dal Lake | Avantipura Ruins | Mughal Gardens ...,Cancellation any time after making the 1st pay...,22485.5,2021,8
2,f060f2954840503cc2fdaf495357b7df,Might of Mewar- Udaipur and Chittorgarh,Luxury,Udaipur|Chittorgarh,2N Udaipur . 1N Chittorgarh,Udaipur|Chittorgarh,2021-04-26,The Ananta:4.4|juSTa Lake Nahargarh Palace:4,New Delhi,IndiGo,0,4,Lake Pichola | Jag Mandir Palace | Saheliyon ...,Cancellation any time after making the 1st pay...,12421.5,2021,4
3,32a19a6c171e67448f2346da46c619dc,Colorful Kerala ( Romantic Getaway ),Premium,Munnar|Kumarakom|Allepey|Kovalam and Poovar,2N Munnar . 1N Kumarakom . 1N Allepey . 2N Kov...,Munnar|Kumarakom|Allepey|Kovalam and Poovar,2021-08-27,Elixir Hills Suites Resort & Spa-MMT Holidays ...,New Delhi,IndiGo,0,5,Mattupetty Dam | Echo Point | Tata Tea Museum...,Cancellation any time after making the 1st pay...,35967.0,2021,8
4,107b068aa0ca03bc6248966f594d105f,A Week In Bangkok & Pattaya,Premium,Pattaya|Bangkok,4N Pattaya . 3N Bangkok,Pattaya|Bangkok,2021-12-12,Dusit Thani Pattaya - MMT Special:4.5|Amari Wa...,New Delhi,Spicejet|Go Air,0,5,"Coral Island Tour with Indian Lunch, Join Spe...",Cancellation any time after making the 1st pay...,25584.0,2021,12


In [ ]:
# select numerical and categorical features
def divideFeatures(train):
    numerical_features = train.select_dtypes(include=[np.number])
    categorical_features = train.select_dtypes(include=[np.object])
    return numerical_features, categorical_features

In [ ]:
continuous_data, categorical_data = divideFeatures(train)
categorical_data[:5]

,Uniq Id,Package Name,Package Type,Destination,Itinerary,Places Covered,Hotel Details,Start City,Airline,Sightseeing Places Covered,Cancellation Rules
0,e788ab76d9d8cf1e6ed2f139645ca5d1,Best of Shimla and Manali Holiday from Delhi,Standard,New Delhi|Shimla|Manali|Chandigarh,1N New Delhi . 2N Shimla . 2N Manali . 1N Chan...,New Delhi|Shimla|Manali|Chandigarh,Not Available,Mumbai,Not Available,Not Available,Not Available
1,178f892630ce3e335a5a41d5d83937fd,Kashmir Valley vacation,Premium,Srinagar|Pahalgam|Srinagar,1N Srinagar . 2N Pahalgam . 1N Srinagar,Srinagar|Pahalgam|Srinagar,The Orchard Retreat & Spa:4.6|WelcomHotel Pine...,New Delhi,IndiGo|IndiGo,Dal Lake | Avantipura Ruins | Mughal Gardens ...,Cancellation any time after making the 1st pay...
2,f060f2954840503cc2fdaf495357b7df,Might of Mewar- Udaipur and Chittorgarh,Luxury,Udaipur|Chittorgarh,2N Udaipur . 1N Chittorgarh,Udaipur|Chittorgarh,The Ananta:4.4|juSTa Lake Nahargarh Palace:4,New Delhi,IndiGo,Lake Pichola | Jag Mandir Palace | Saheliyon ...,Cancellation any time after making the 1st pay...
3,32a19a6c171e67448f2346da46c619dc,Colorful Kerala ( Romantic Getaway ),Premium,Munnar|Kumarakom|Allepey|Kovalam and Poovar,2N Munnar . 1N Kumarakom . 1N Allepey . 2N Kov...,Munnar|Kumarakom|Allepey|Kovalam and Poovar,Elixir Hills Suites Resort & Spa-MMT Holidays ...,New Delhi,IndiGo,Mattupetty Dam | Echo Point | Tata Tea Museum...,Cancellation any time after making the 1st pay...
4,107b068aa0ca03bc6248966f594d105f,A Week In Bangkok & Pattaya,Premium,Pattaya|Bangkok,4N Pattaya . 3N Bangkok,Pattaya|Bangkok,Dusit Thani Pattaya - MMT Special:4.5|Amari Wa...,New Delhi,Spicejet|Go Air,"Coral Island Tour with Indian Lunch, Join Spe...",Cancellation any time after making the 1st pay...


In [ ]:
df_train = train.drop(columns=['Uniq Id'])
print('Shape',df_train.shape)
df_train.head(2)

Shape (21000, 16)


,Package Name,Package Type,Destination,Itinerary,Places Covered,Travel Date,Hotel Details,Start City,Airline,Flight Stops,Meals,Sightseeing Places Covered,Cancellation Rules,Per Person Price,Year,month
0,Best of Shimla and Manali Holiday from Delhi,Standard,New Delhi|Shimla|Manali|Chandigarh,1N New Delhi . 2N Shimla . 2N Manali . 1N Chan...,New Delhi|Shimla|Manali|Chandigarh,2021-07-30,Not Available,Mumbai,Not Available,2,3,Not Available,Not Available,11509.0,2021,7
1,Kashmir Valley vacation,Premium,Srinagar|Pahalgam|Srinagar,1N Srinagar . 2N Pahalgam . 1N Srinagar,Srinagar|Pahalgam|Srinagar,2021-08-12,The Orchard Retreat & Spa:4.6|WelcomHotel Pine...,New Delhi,IndiGo|IndiGo,0,5,Dal Lake | Avantipura Ruins | Mughal Gardens ...,Cancellation any time after making the 1st pay...,22485.5,2021,8


In [ ]:
continuous_data, categorical_data = divideFeatures(df_train)
categorical_data[:5]

,Package Name,Package Type,Destination,Itinerary,Places Covered,Hotel Details,Start City,Airline,Sightseeing Places Covered,Cancellation Rules
0,Best of Shimla and Manali Holiday from Delhi,Standard,New Delhi|Shimla|Manali|Chandigarh,1N New Delhi . 2N Shimla . 2N Manali . 1N Chan...,New Delhi|Shimla|Manali|Chandigarh,Not Available,Mumbai,Not Available,Not Available,Not Available
1,Kashmir Valley vacation,Premium,Srinagar|Pahalgam|Srinagar,1N Srinagar . 2N Pahalgam . 1N Srinagar,Srinagar|Pahalgam|Srinagar,The Orchard Retreat & Spa:4.6|WelcomHotel Pine...,New Delhi,IndiGo|IndiGo,Dal Lake | Avantipura Ruins | Mughal Gardens ...,Cancellation any time after making the 1st pay...
2,Might of Mewar- Udaipur and Chittorgarh,Luxury,Udaipur|Chittorgarh,2N Udaipur . 1N Chittorgarh,Udaipur|Chittorgarh,The Ananta:4.4|juSTa Lake Nahargarh Palace:4,New Delhi,IndiGo,Lake Pichola | Jag Mandir Palace | Saheliyon ...,Cancellation any time after making the 1st pay...
3,Colorful Kerala ( Romantic Getaway ),Premium,Munnar|Kumarakom|Allepey|Kovalam and Poovar,2N Munnar . 1N Kumarakom . 1N Allepey . 2N Kov...,Munnar|Kumarakom|Allepey|Kovalam and Poovar,Elixir Hills Suites Resort & Spa-MMT Holidays ...,New Delhi,IndiGo,Mattupetty Dam | Echo Point | Tata Tea Museum...,Cancellation any time after making the 1st pay...
4,A Week In Bangkok & Pattaya,Premium,Pattaya|Bangkok,4N Pattaya . 3N Bangkok,Pattaya|Bangkok,Dusit Thani Pattaya - MMT Special:4.5|Amari Wa...,New Delhi,Spicejet|Go Air,"Coral Island Tour with Indian Lunch, Join Spe...",Cancellation any time after making the 1st pay...


In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()

In [ ]:
for col in categorical_data:
  df_train[col] = lb.fit_transform(df_train[col])

In [ ]:
print('Shape:',df_train.shape)
df_train.head(3)

Shape: (21000, 16)


,Package Name,Package Type,Destination,Itinerary,Places Covered,Travel Date,Hotel Details,Start City,Airline,Flight Stops,Meals,Sightseeing Places Covered,Cancellation Rules,Per Person Price,Year,month
0,421,4,417,214,417,2021-07-30,3202,0,211,2,3,1713,5,11509.0,2021,7
1,1138,3,533,283,533,2021-08-12,5463,1,125,0,5,524,0,22485.5,2021,8
2,1417,2,549,700,549,2021-04-26,5092,1,114,0,4,1000,0,12421.5,2021,4


In [ ]:
X = df_train.drop(columns=['Per Person Price','Travel Date','month','Package Type','Meals','Destination','Places Covered'])
print('Shape:',X.shape)
X.head(2)

Shape: (21000, 9)


,Package Name,Itinerary,Hotel Details,Start City,Airline,Flight Stops,Sightseeing Places Covered,Cancellation Rules,Year
0,421,214,3202,0,211,2,1713,5,2021
1,1138,283,5463,1,125,0,524,0,2021


In [ ]:
df_train['Per Person Price'] = np.log(df_train['Per Person Price'])

In [ ]:
y=df_train['Per Person Price']

In [ ]:
y

0         9.350885
1        10.020626
2         9.427184
3        10.490357
4        10.149722
           ...    
20995     9.690820
20996    10.427343
20997    10.307752
20998    10.245143
20999     9.663706
Name: Per Person Price, Length: 21000, dtype: float64

In [ ]:
from sklearn.preprocessing import StandardScaler,RobustScaler
rs = StandardScaler()
for c in X.columns:
    X[c] = rs.fit_transform(X[c].values.reshape(-1, 1))
X.head()

,Package Name,Itinerary,Hotel Details,Start City,Airline,Flight Stops,Sightseeing Places Covered,Cancellation Rules,Year
0,-1.002439,-1.095036,0.061264,-1.374691,0.599893,1.353001,1.423497,1.207148,-0.205638
1,0.125036,-0.840970,1.427552,0.727436,-0.623436,-1.042296,-0.929885,-0.430075,-0.205638
2,0.563760,0.694473,1.203362,0.727436,-0.779908,-1.042296,0.012260,-0.430075,-0.205638
3,-0.900227,0.057466,-1.350344,0.727436,-0.779908,-1.042296,0.198313,-0.430075,-0.205638
4,-1.356249,1.508220,-1.376328,0.727436,0.983962,-1.042296,-1.301992,-0.430075,-0.205638


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2,random_state=30)

In [ ]:
!pip install optuna

In [ ]:
import optuna
from math import sqrt
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_valid = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [ ]:
def objective(trial):    
    params = {
            'num_leaves': trial.suggest_int('num_leaves', 22, 712),
            'max_bin': trial.suggest_int('max_bin', 300, 1400),
            'objective': 'regression',
            'metric': 'RMSE',
            'learning_rate': trial.suggest_float('learning_rate',0.005,0.2),
            'random_state' : 33,
            'max_depth': trial.suggest_int('max_depth', 4, 16),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 21),
            'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
            'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 9),
            'min_child_samples': trial.suggest_int('min_child_samples', 3, 100),
            'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 1.0),
            'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 1.0),
            'early_stopping_rounds': 10
            }
    
    model = lgb.train(params, lgb_train, valid_sets=[lgb_valid], verbose_eval=100, num_boost_round=800)
    y_pred  = model.predict(X_test)
    score = np.sqrt(mean_squared_log_error(y_test, y_pred))
    print(f"RMSE = {score}")
    return score

In [ ]:
study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials = 50)

[I 2021-04-08 18:14:50,261] A new study created in memory with name: no-name-4bffd2ab-0cf9-4a67-9c29-93619d368f6b
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning:

Found `early_stopping_rounds` in params. Will use it instead of argument



Training until validation scores don't improve for 10 rounds.
[100]	valid_0's rmse: 0.273733
[200]	valid_0's rmse: 0.250601
[300]	valid_0's rmse: 0.238523
[400]	valid_0's rmse: 0.230386
[500]	valid_0's rmse: 0.225057
[600]	valid_0's rmse: 0.220761
[700]	valid_0's rmse: 0.217141
[800]	valid_0's rmse: 0.214122
Did not meet early stopping. Best iteration is:
[800]	valid_0's rmse: 0.214122


[I 2021-04-08 18:14:53,697] Trial 0 finished with value: 0.019824811444297083 and parameters: {'num_leaves': 609, 'max_bin': 1191, 'learning_rate': 0.1706551003549254, 'max_depth': 5, 'min_child_weight': 7, 'feature_fraction': 0.7704519771250012, 'bagging_fraction': 0.9662504747685563, 'bagging_freq': 3, 'min_child_samples': 81, 'lambda_l1': 0.001501684762881629, 'lambda_l2': 6.587557895433719e-07}. Best is trial 0 with value: 0.019824811444297083.


RMSE = 0.019824811444297083
Training until validation scores don't improve for 10 rounds.


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning:

Found `early_stopping_rounds` in params. Will use it instead of argument



...

RMSE = 0.016954249416854254
Training until validation scores don't improve for 10 rounds.


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning:

Found `early_stopping_rounds` in params. Will use it instead of argument



[100]	valid_0's rmse: 0.204134
[200]	valid_0's rmse: 0.189963
[300]	valid_0's rmse: 0.185936
[400]	valid_0's rmse: 0.184373
[500]	valid_0's rmse: 0.183568
Early stopping, best iteration is:
[498]	valid_0's rmse: 0.1835


[I 2021-04-08 18:19:24,918] Trial 49 finished with value: 0.016925916856715472 and parameters: {'num_leaves': 237, 'max_bin': 1109, 'learning_rate': 0.07197448740086385, 'max_depth': 16, 'min_child_weight': 8, 'feature_fraction': 0.5497524289695054, 'bagging_fraction': 0.9970080341459777, 'bagging_freq': 1, 'min_child_samples': 8, 'lambda_l1': 0.10323861609376331, 'lambda_l2': 0.007450028014742337}. Best is trial 49 with value: 0.016925916856715472.


RMSE = 0.016925916856715472


In [ ]:
print(f"Trials : {len(study.trials)}")
print(f"Best Trial : {study.best_trial}")
best = study.best_trial
for i, j in best.params.items():
    print(f"{i} : {j}")
print(f"{best.value}")

Trials : 50
Best Trial : FrozenTrial(number=49, values=[0.016925916856715472], datetime_start=datetime.datetime(2021, 4, 8, 18, 19, 17, 363757), datetime_complete=datetime.datetime(2021, 4, 8, 18, 19, 24, 917323), params={'num_leaves': 237, 'max_bin': 1109, 'learning_rate': 0.07197448740086385, 'max_depth': 16, 'min_child_weight': 8, 'feature_fraction': 0.5497524289695054, 'bagging_fraction': 0.9970080341459777, 'bagging_freq': 1, 'min_child_samples': 8, 'lambda_l1': 0.10323861609376331, 'lambda_l2': 0.007450028014742337}, distributions={'num_leaves': IntUniformDistribution(high=712, low=22, step=1), 'max_bin': IntUniformDistribution(high=1400, low=300, step=1), 'learning_rate': UniformDistribution(high=0.2, low=0.005), 'max_depth': IntUniformDistribution(high=16, low=4, step=1), 'min_child_weight': IntUniformDistribution(high=21, low=1, step=1), 'feature_fraction': UniformDistribution(high=1.0, low=0.4), 'bagging_fraction': UniformDistribution(high=1.0, low=0.4), 'bagging_freq': IntUn

In [ ]:
best_params = best.params

In [ ]:
best_params

{'bagging_fraction': 0.9970080341459777,
 'bagging_freq': 1,
 'feature_fraction': 0.5497524289695054,
 'lambda_l1': 0.10323861609376331,
 'lambda_l2': 0.007450028014742337,
 'learning_rate': 0.07197448740086385,
 'max_bin': 1109,
 'max_depth': 16,
 'min_child_samples': 8,
 'min_child_weight': 8,
 'num_leaves': 237}

In [ ]:
import lightgbm as lgb
clf = lgb.LGBMRegressor(bagging_fraction=0.9970080341459777,bagging_freq=1,
 feature_fraction=0.5497524289695054,
 lambda_l1= 0.10323861609376331,
 lambda_l2= 0.007450028014742337,
 learning_rate= 0.07197448740086385,
 max_bin= 1109,
 max_depth= 16,
 min_child_samples= 8,
 min_child_weight=8,
 num_leaves=237)
clf.fit(X_train, y_train)

LGBMRegressor(bagging_fraction=0.9970080341459777, bagging_freq=1,
              boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              feature_fraction=0.5497524289695054, importance_type='split',
              lambda_l1=0.10323861609376331, lambda_l2=0.007450028014742337,
              learning_rate=0.07197448740086385, max_bin=1109, max_depth=16,
              min_child_samples=8, min_child_weight=8, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=237, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
pred = clf.predict(X_test)

from sklearn.metrics import r2_score
print(r2_score(y_test, pred))

0.8630614747169733


In [ ]:
from sklearn.metrics import mean_squared_log_error
np.sqrt(mean_squared_log_error(y_test, pred))

0.018719252235271742

In [ ]:
print('Shape:',test.shape)
test.head(2)

Shape: (9000, 14)


,Uniq Id,Package Name,Package Type,Destination,Itinerary,Places Covered,Travel Date,Hotel Details,Start City,Airline,Flight Stops,Meals,Sightseeing Places Covered,Cancellation Rules
0,12c47aefbcca65d6156ba0bc7946704b,Goa 5N stay with flights & Transfers-North Goa,Luxury,Goa,5N Goa,Goa,13-07-2021,Novotel Goa Resort & Spa:4.1,New Delhi,IndiGo,0,4,Not Available,Cancellation any time after making the 1st pay...
1,156febf7d27315ae9f210ed13bca3ac2,Into the Wild - Weekend at Bandipur National Park,Luxury,Bandipur,2N Bandipur,Bandipur,20-03-2021,The Windflower Jungle Resorts & Spa Bandipur:4.4,New Delhi,Not Available,0,4,Bandipur National Park,This cancellation policy is applicable for boo...


In [ ]:
test['Travel Date'] = pd.to_datetime(test['Travel Date'])

In [ ]:
test['Year']=pd.DatetimeIndex(test['Travel Date']).year
test['month']=pd.DatetimeIndex(test['Travel Date']).month


In [ ]:
df_test = test.drop(columns=['Uniq Id'])
print('Shape',df_test.shape)
df_test.head(2)

Shape (9000, 15)


,Package Name,Package Type,Destination,Itinerary,Places Covered,Travel Date,Hotel Details,Start City,Airline,Flight Stops,Meals,Sightseeing Places Covered,Cancellation Rules,Year,month
0,Goa 5N stay with flights & Transfers-North Goa,Luxury,Goa,5N Goa,Goa,2021-07-13,Novotel Goa Resort & Spa:4.1,New Delhi,IndiGo,0,4,Not Available,Cancellation any time after making the 1st pay...,2021,7
1,Into the Wild - Weekend at Bandipur National Park,Luxury,Bandipur,2N Bandipur,Bandipur,2021-03-20,The Windflower Jungle Resorts & Spa Bandipur:4.4,New Delhi,Not Available,0,4,Bandipur National Park,This cancellation policy is applicable for boo...,2021,3


In [ ]:
continuous_data, categorical_data = divideFeatures(df_test)
categorical_data[:5]

,Package Name,Package Type,Destination,Itinerary,Places Covered,Hotel Details,Start City,Airline,Sightseeing Places Covered,Cancellation Rules
0,Goa 5N stay with flights & Transfers-North Goa,Luxury,Goa,5N Goa,Goa,Novotel Goa Resort & Spa:4.1,New Delhi,IndiGo,Not Available,Cancellation any time after making the 1st pay...
1,Into the Wild - Weekend at Bandipur National Park,Luxury,Bandipur,2N Bandipur,Bandipur,The Windflower Jungle Resorts & Spa Bandipur:4.4,New Delhi,Not Available,Bandipur National Park,This cancellation policy is applicable for boo...
2,Unforgettable Munnar Getaway - Private Picnic ...,Standard,Munnar,3N Munnar,Munnar,Not Available,New Delhi,Not Available,Old Munnar Town | Eravikulam national park | ...,This cancellation policy is applicable for boo...
3,Munnar for Adventure Lovers - Top Valley Trek,Deluxe,Munnar,3N Munnar,Munnar,Not Available,Mumbai,Not Available,Tata Tea Museum | Eravikulam national park | ...,This cancellation policy is applicable for boo...
4,Splendours of Kerala - Make your own Tea,Luxury,Cochin|Munnar|Thekkady|Allepey,1N Cochin . 2N Munnar . 1N Thekkady . 1N Allepey,Cochin|Munnar|Thekkady|Allepey,Casino Hotel - Cgh Earth-MMT Holidays Special:...,New Delhi,IndiGo,Fort Cochin | Dutch Palace | Jewish Synagogue...,Cancellation any time after making the 1st pay...


In [ ]:
for col in categorical_data:
  df_test[col] = lb.fit_transform(df_test[col])

In [ ]:
X = df_test.drop(columns=['Travel Date','month','Package Type','Meals','Destination','Places Covered'])
print('Shape:',X.shape)
X.head(2)

Shape: (9000, 9)


,Package Name,Itinerary,Hotel Details,Start City,Airline,Flight Stops,Sightseeing Places Covered,Cancellation Rules,Year
0,797,918,1959,1,80,0,1475,0,2021
1,1003,308,3442,1,147,0,66,6,2021


In [ ]:
from sklearn.preprocessing import StandardScaler,RobustScaler
rs = StandardScaler()
for c in X.columns:
    X[c] = rs.fit_transform(X[c].values.reshape(-1, 1))
X.head()

,Package Name,Itinerary,Hotel Details,Start City,Airline,Flight Stops,Sightseeing Places Covered,Cancellation Rules,Year
0,-0.338273,1.590855,0.077686,0.719340,-0.771121,-1.046411,1.472236,-0.442660,-0.198446
1,0.008203,-0.721739,1.555609,0.719340,0.614940,-1.046411,-1.743153,2.420738,-0.198446
2,1.637987,1.075261,0.072703,0.719340,0.614940,1.329096,0.395115,2.420738,5.039143
3,0.606968,1.075261,0.072703,-1.390162,0.614940,0.141343,1.127648,2.420738,-0.198446
4,1.345332,-1.650568,-1.684261,0.719340,-0.771121,-1.046411,-0.574751,-0.442660,-0.198446


In [ ]:
prediction = clf.predict(X)
prediction

array([ 9.92886743,  9.88525168,  9.68037783, ..., 10.02411976,
        9.9214275 ,  9.61595485])

In [ ]:
Predict=np.exp(prediction)

In [ ]:
submission = pd.DataFrame({'Per Person Price':Predict})
submission.head()

,Per Person Price
0,20514.093709
1,19638.587839
2,16000.541319
3,17886.563602
4,22578.753990


In [ ]:
submission.to_csv('lgbm-o.csv')